In [2]:
cd ../

/Users/arbaaz/Downloads/mabe


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd

train_metadata = pd.read_csv("/Users/arbaaz/Downloads/mabe/ds/train.csv")
train_metadata.head()

,lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,...,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,AdaptableSnail,44566106,CD-1 (ICR),white,male,10.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1228,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
1,AdaptableSnail,143861384,CD-1 (ICR),white,male,3.0,8-12 weeks,NaN,CD-1 (ICR),white,...,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2,AdaptableSnail,209576908,CD-1 (ICR),white,male,7.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1266,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
3,AdaptableSnail,278643799,CD-1 (ICR),white,male,11.0,8-12 weeks,wireless device,CD-1 (ICR),white,...,16.0,1224,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
4,AdaptableSnail,351967631,CD-1 (ICR),white,male,14.0,8-12 weeks,NaN,CD-1 (ICR),white,...,16.0,1204,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut


In [5]:
train_metadata["frames_per_second"].mean()

np.float64(30.402647627716462)

In [ ]:
import pandas as pd

# fps stats
rows = []
for lab_id, grp in train_metadata.groupby("lab_id"):
    rows.append({
        "lab_id": lab_id,
        "mean_fps": grp["frames_per_second"].mean(),
        "min_fps": grp["frames_per_second"].min(),
        "max_fps": grp["frames_per_second"].max(),
    })

# create dataframe
fps_stats = pd.DataFrame(rows).set_index("lab_id")

# display
fps_stats


,mean_fps,min_fps,max_fps
lab_id,,,
AdaptableSnail,28.823529,25.00,30.00
BoisterousParrot,30.000000,30.00,30.00
CRIM13,25.000000,25.00,25.00
CalMS21_supplemental,30.000000,30.00,30.00
CalMS21_task1,30.000000,30.00,30.00
CalMS21_task2,30.000000,30.00,30.00
CautiousGiraffe,25.000000,25.00,25.00
DeliriousFly,60.008333,60.00,60.01
ElegantMink,30.000000,30.00,30.00


In [13]:
train_metadata["video_duration_sec"].mean() / 60

np.float64(2.2441309212272915)

In [ ]:
import pandas as pd

# video durations in mins
rows = []
for lab_id, grp in train_metadata.groupby("lab_id"):
    rows.append({
        "lab_id": lab_id,
        "mean_min": grp["video_duration_sec"].mean() / 60,
        "min_min": grp["video_duration_sec"].min() / 60,
        "max_min": grp["video_duration_sec"].max() / 60,
    })

# create dataframe
sec_stats = pd.DataFrame(rows).set_index("lab_id")

# display
sec_stats

,mean_min,min_min,max_min
lab_id,,,
AdaptableSnail,26.313725,10.001667,59.983333
BoisterousParrot,330.014167,330.011667,330.016667
CRIM13,6.622381,0.596000,11.080667
CalMS21_supplemental,10.371487,2.025000,20.936667
CalMS21_task1,5.396749,0.316667,36.408333
CalMS21_task2,10.901447,3.610000,17.355000
CautiousGiraffe,10.000000,10.000000,10.000000
DeliriousFly,14.999167,14.996667,15.001667
ElegantMink,26.618158,14.761667,30.796667


In [28]:
from collections import defaultdict

total_samples = defaultdict(lambda: defaultdict(float))

for lab_id, grp in train_metadata.groupby("lab_id"):
    for idx, row in grp.iterrows():
        total_samples[lab_id][idx] = row["frames_per_second"] * row["video_duration_sec"]

summary = []
for lab_id, samples in total_samples.items():
    values = list(samples.values())
    summary.append({
        "lab_id": lab_id,
        "sum_total_samples": sum(values),
        "min_total_samples": min(values),
        "max_total_samples": max(values)
    })

# Step 3: convert to DataFrame
summary_df = pd.DataFrame(summary).set_index("lab_id")

# Display nicely
summary_df

,sum_total_samples,min_total_samples,max_total_samples
lab_id,,,
AdaptableSnail,733220.000,18003.000,89975.000
BoisterousParrot,4752204.000,594021.000,594030.000
CRIM13,208605.000,894.000,16621.000
CalMS21_supplemental,5544597.000,3645.000,37686.000
CalMS21_task1,981129.000,570.000,65535.000
CalMS21_task2,1491318.000,6498.000,31239.000
CautiousGiraffe,150000.000,15000.000,15000.000
DeliriousFly,324026.999,53988.000,54015.001
ElegantMink,910341.000,26571.000,55434.000


In [ ]:
# flatten all numeric values across all labs
global_total_samples = sum(
    value
    for lab_dict in total_samples.values()
    for value in lab_dict.values()
)

print(f"Global total samples: {global_total_samples / 1e6}")

Global total samples: 35.616097743
